In [1]:
from pathlib import Path
import os
import json

# DATA_BASE_DIR = Path("/data/vision/polina/projects/wmh/dhollidt/datasets/toybox-5")
DATA_BASE_DIR = Path("/data/vision/polina/projects/wmh/dhollidt/datasets/klevr_nesf")
# DATA_BASE_DIR = Path("/data/vision/polina/projects/wmh/dhollidt/kubric_datasets/klevr")
# MODEL_BASE_DIR = Path("/data/vision/polina/projects/wmh/dhollidt/documents/nerf/klever_depth_models_nesf")
MODEL_BASE_DIR = Path("/data/vision/polina/projects/wmh/dhollidt/documents/nerf/klever_depth_normal_models_nesf")
# MODEL_BASE_DIR = Path("/data/vision/polina/projects/wmh/dhollidt/documents/nerf/toybox-5-depth-normal-models")
# MODEL_BASE_DIR = Path("/data/vision/polina/projects/wmh/dhollidt/documents/nerf/toybox-5-depth-normal-models-nesf-2")
# MODEL_BASE_DIR = Path("/data/vision/polina/projects/wmh/dhollidt/documents/nerf/klevr-normal")

TRAIN_START = 0
TEST_START = 100

MODEL_NAME="depth-nerfacto"
TIMESTAMP = "07_04_23"

def get_highest_load_step(load_dir: Path):
    load_step = sorted(int(x[x.find("-") + 1 : x.find(".")]) for x in os.listdir(load_dir))[-1]
    return load_step
    

def create_nesf_data_config(num_scenes: int, split: str, out_path: Path, split_ratio = None):
    start = TRAIN_START if split == "train" else TEST_START
    
    
    config = []
    for scene_idx in range(start, start+num_scenes):
        load_dir = MODEL_BASE_DIR / str(scene_idx) / MODEL_NAME / TIMESTAMP / "nerfstudio_models"
        scene_config = {
            "model_config": {
                # "is_euclidean_depth": True,
                # "max_res": 1536,
                # "depth_sigma": 0.001,
                # "depth_loss_mult": 0.01
                },
            "data_parser_config": {
                "data": str(DATA_BASE_DIR / str(scene_idx)),
            },
            "load_step": get_highest_load_step(load_dir),
            "load_dir": str(load_dir),
            }
        
        if split == "test":
            scene_config["data_parser_config"]["train_split_percentage"] = 0.0
            scene_config["set_type"] = "test"
            
        if split_ratio is not None:
            scene_config["data_parser_config"]["train_split_percentage"] = split_ratio

        config.append(scene_config)
        
    data_config = {"config": config}
    
    # write to file
    with open(out_path, 'w') as f:
        json.dump(data_config, f, indent=4)
    
    

In [4]:
TOATAL_NUM_IMAGES_PER_SCENE = 300.0
TRAIN_IMAGES_TO_USE = int(TOATAL_NUM_IMAGES_PER_SCENE * 0.9) # default
# TRAIN_IMAGES_TO_USE = 10

split_ratio =  TRAIN_IMAGES_TO_USE / TOATAL_NUM_IMAGES_PER_SCENE
TRAIN_SCENES = 10
create_nesf_data_config(TRAIN_SCENES, "train", Path(f"/data/vision/polina/projects/wmh/dhollidt/documents/nerf/data/klever_depth_normal_nesf_train_{TRAIN_SCENES}_{TRAIN_IMAGES_TO_USE}.json"), split_ratio=split_ratio)
TEST_SCNES = 17
create_nesf_data_config(TEST_SCNES, "test", Path(f"/data/vision/polina/projects/wmh/dhollidt/documents/nerf/data/klever_depth_normal_nesf_test_{TEST_SCNES}.json"))

In [3]:
TOATAL_NUM_IMAGES_PER_SCENE = 300.0
TRAIN_IMAGES_TO_USE = int(300 * 0.9) # default
# TRAIN_IMAGES_TO_USE = 10

split_ratio =  TRAIN_IMAGES_TO_USE / TOATAL_NUM_IMAGES_PER_SCENE
TRAIN_SCENES = 80
create_nesf_data_config(TRAIN_SCENES, "train", Path(f"/data/vision/polina/projects/wmh/dhollidt/documents/nerf/data/toybox-5_nesf_2_train_{TRAIN_SCENES}_{TRAIN_IMAGES_TO_USE}.json"), split_ratio=split_ratio)
TEST_SCNES = 17
create_nesf_data_config(TEST_SCNES, "test", Path(f"/data/vision/polina/projects/wmh/dhollidt/documents/nerf/data/toybox-5_nesf_2_test_{TEST_SCNES}.json"))